# Projet 7 - Deployment

In [1]:
import requests
import os
script_folder = os.path.join(os.getcwd(), "source_dir")
os.makedirs(script_folder, exist_ok=True)

In [2]:
from azureml.core import Workspace, Dataset

subscription_id = 'your_subscription_ID'
resource_group = 'OC-P7'
workspace_name = 'P7_ML'

ws = Workspace(subscription_id, resource_group, workspace_name)

## Dummy entry script

In [3]:
%%writefile $script_folder/echo_score.py

import json

def init():
    print('This is init')

def run(data):
    test = json.loads(data)
    print(f'received data {test}')
    return(f'test is {test}')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/oc-p7-ml-ds3/code/Users/yann.hereng.openclassrooms/source_dir/echo_score.py


# Local deployment for test and debug

## Inference configuration

In [4]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='project_environment')
inf_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./echo_score.py')

## Deployment configuration

In [5]:
from azureml.core.webservice import LocalWebservice

deploy_config = LocalWebservice.deploy_configuration(port=6789)

## Local Deploy

In [7]:
from azureml.core.model import Model


In [8]:
model = Model(workspace=ws,name='LSTM_Glove')

In [8]:
from azureml.core.model import Model

service = Model.deploy(ws, "myservice", [model] , inf_config, deploy_config)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

KeyboardInterrupt: 

## Test with a query

In [12]:
import requests
import json

uri = service.scoring_uri
requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}


# Final entry script

In [10]:
%%writefile $script_folder/score.py

import os
import pandas as pd
import json
import keras
import nltk

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.preprocessing import FunctionTransformer
from azureml.core.model import Model


import json

def init():
    print('***************This is init************************************')
    global model # variable needs to be global to be used in run()
    model = keras.models.load_model(Model.get_model_path('LSTM_Glove'))

def run(request):
    #Load data
    print(request)    
    data_request = pd.read_json(request)
    X=tokenizer(data_request['text']).str.join(sep=' ')

    #Run inference
    y_pred = model.predict(X)
    print(y_pred)
    return (y_pred.tolist()) # the object returned needs to be serialisable

def tokenizer(data, y = None):
    '''tokenizes input dataframe considering words of 2 and more characters
       and lowercase text and remove numbers
    
       Parameters
       ----------
       data : Pandas series to tokenize       
       Returns
       --------
       Pandas series list of tokens'''
                      
    tokenizer = nltk.RegexpTokenizer(r'\w{2,}')
    sentences = data.str.lower()
    sentences = sentences.str.replace('\d+', '',regex=True)
    results = sentences.apply(tokenizer.tokenize)
  
    return results


Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/oc-p7-ml-ds3/code/Users/yann.hereng.openclassrooms/source_dir/score.py


## Deploy localy final script

We'll load an image of predifined environment 'AzureML-TensorFlow-2.2-CPU'

### Create and customize environment

In [11]:
env = Environment.get(workspace=ws, name='AzureML-TensorFlow-2.2-CPU')
curated_clone = env.clone("customize_curated") # clone the environment to customize it


In [12]:
# Install nedded packages
python_packages=['Keras==2.4.3','nltk','sklearn','tensorflow==2.3.0']
for package in python_packages:
    curated_clone.python.conda_dependencies.add_pip_package(package)

### Modify inference config

In [13]:
inf_config = InferenceConfig(environment=curated_clone, source_directory='./source_dir', entry_script='./score.py')

In [13]:
from azureml.core.model import Model


service = Model.deploy(ws, "myservice", [model], inf_config, deploy_config)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 5a5bf34d87e743cba82cd0b93b4b143b.azurecr.io
Logging into Docker registry 5a5bf34d87e743cba82cd0b93b4b143b.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 5a5bf34d87e743cba82cd0b93b4b143b.azurecr.io/azureml/azureml_44bf330ff8c523d2cf00804f469f8f13
 ---> 2301ff468a77
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> a5838bafaecd
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6Ijg3ODE3NTFjLTcwZGQtNDQxYS04YzQ1LTIyNzQyMDg4NTFjMCIsInJlc291cmNlR3JvdXBOYW1lIjoib2MtcDciLCJhY2NvdW50TmFtZSI6InA3X21sIiwid29ya3NwYWNlSWQiOiI1YTViZjM0ZC04N2U3LTQzY2ItYTgyYy1kMGI5M2I0YjE0M2IifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in ff4bb9db1829
 ---> 7b345a270a49
Step 4/5 : RUN mv '/var/azureml-app/tmpxjuwyrgi.py' /var/azureml-app/main.py
 ---> Running in b6d57e64c588
 ---> 25887629e359

use service.reload() when modifying script without restarting the whole service

requests.get('http://localhost:6789') to test liveness of service

print(service.get_logs()) to see logs and debug



In [20]:
service.reload()

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.


In [14]:
import json

In [15]:
request = {"target":{"1516037":1,"589923":0,"213819":0,"10047":0,"1330460":1},
        "text":{"1516037":"Wow, its later than I feel, better wrap up another twitteriffic day. night -all  ",
                 "589923":"@lemonissimo I think the reason I twitted so much in Italy, there was so much to do &amp; see! Here it's just working through videos... ",
                 "213819":"@GericaQuinn ahhhhh! dude u suck! lmao. jk! but i love my little davidkins! haha. i wanna pop n' lock on him  lol",
                 "10047":"...aaaand there goes that great day  RIP Mrs Wever","1330460":"another morning joe free morning ahhhh ... sun ... birds singing ... nice! ",
                 "363111":"@cdouglasroberts.... awww  Im across the state in Knoxville."}}
request=json.dumps(request)

In [23]:
uri = service.scoring_uri
headers = {'Content-Type': 'application/json'}
response = requests.post(uri, data=request,headers=headers)
response

<Response [200]>

In [18]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-05-13T16:46:30,611237936+00:00 - nginx/run 
/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-05-13T16:46:30,615961254+00:00 - rsyslog/run 
/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-05-13T16:46:30,618660064+00:00 - iot-server/run 
bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by bash)
2021-05-13T16:46:30,622485078+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
/bin/bash: /azureml

In [23]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_426e3ece822f802c0fe0da3258bc8612/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_426e3ece822f802c0fe0da3258bc8612/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_426e3ece822f802c0fe0da3258bc8612/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-05-12T10:48:13,950909430+00:00 - iot-server/run 
2021-05-12T10:48:13,953190764+00:00 - nginx/run 
/bin/bash: /azureml-envs/azureml_426e3ece822f802c0fe0da3258bc8612/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2021-05-12T10:48:13,959343358+00:00 - gunicorn/run 
2021-05-12T10:48:13,962803010+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_426e3ece822f802c0fe0da3258bc8612/lib/libtinfo.so.5: no version information available (required by bash)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
/bin/bash: /azureml

In [24]:
response.json()

[[0.6983094811439514],
 [0.4235050082206726],
 [0.6183003783226013],
 [0.2214709222316742],
 [0.9137091636657715],
 [0.11121213436126709]]

## Deploy in the cloud

### use ACI for deployment config

In [16]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, memory_gb = 2)

In [17]:
service = Model.deploy(ws, "myservice",[model], inference_config=inf_config, deployment_config=deployment_config )
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-13 17:15:58+00:00 Creating Container Registry if not exists.
2021-05-13 17:15:58+00:00 Registering the environment.
2021-05-13 17:16:00+00:00 Use the existing image.
2021-05-13 17:16:00+00:00 Generating deployment configuration.
2021-05-13 17:16:02+00:00 Submitting deployment to compute..
2021-05-13 17:16:10+00:00 Checking the status of deployment myservice..
2021-05-13 17:18:35+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_8fb2b84aef302194159e21d427136a88/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/b

In [18]:
from azureml.core import Webservice

service = Webservice(workspace=ws, name='myservice')
scoring_uri = service.scoring_uri

In [19]:
scoring_uri

'http://24d13703-5fe9-40cd-acd4-efeace756110.westeurope.azurecontainer.io/score'

In [21]:
resp = requests.post(scoring_uri, data=request, headers=headers)

NameError: name 'headers' is not defined

In [28]:
resp

<Response [200]>

In [29]:
resp.json()

[[0.6778625249862671],
 [0.3400944471359253],
 [0.7227505445480347],
 [0.2214709222316742],
 [0.9137091636657715],
 [0.11121213436126709]]